In [0]:
!pip install sobol_seq
import numpy as np
import math
from scipy.special import erf, erfc
import itertools
import sobol_seq

# a=alpha/alpha_obs, b=beta/beta_obs, c=gamma/gamma_obs, l=lambda
a_min=0.2
a_max=2.07 #without plate tectonics
a_ptmin=137/153
a_ptmax=137/136 #with plate tectonics
b_min=0.0
b_max=1.1/.511
c_min=0.1
c_max=134
l_minus=.0393*a_min**(3/2)*b_max**(-3/4)

#FUNCTIONS
def heaviside(x):
    return np.heaviside(x,1)
def m1(x):
    return min(x,1-x) 
def r2(p):
    return float('%.2g' % p)
def r3(p):
    return float('%.3g' % p)
minn=np.vectorize(min)
def ramp(x):
    return minn(1,x)
def signs(*P_list):
    say=''.join([P_primes[i][P_list[i]] for i in range(len(P_list))])
    if say.strip()=='':
        print('au naturale')
    else:
        print(say)

In [0]:
#STARS

def pmeas(a,b,c): #theory measure
    return 1/b/c
def Nstars(a,b,c): #number of stars in the universe
    return a**(-3/2)*b**(3/4)*c**3

beta_imf=2.35
def l_min(a,b,c): #smallest star capable of H fusion
    return .0393*a**(3/2)*b**(-3/4)
def p_lmin(l,a,b,c): #Salpeter initial mass function
    return (l_min(a,b,c)/l_minus)**(beta_imf-1)*heaviside(l-l_min(a,b,c))
def masch(l,a,b,c): # gives imf a knee
    return 8.9403*(1+(5.06*l_min(a,b,c)/l)**1.35)**-1.4

#TIDAL LOCKING
def l_TL(a,b,c): #tidal locking limit
    return .474*a**(5/2)*b**(1/2)*c**(-4/11)
def l_TLb(a,b,c): #t_TL compared to t_bio
    return .474*a**(47/17)*b**(12/17)*c**(-4/11)
def f_TL(l,a,b,c):
    return heaviside(l-l_TL(a,b,c))

#CONVECTION
def l_conv(a,b,c): #convective stellar mass
    return .194*a**3*b*c**(-1/2)
def f_conv(l,a,b,c):
    return heaviside(l-l_conv(a,b,c))

#BIOLOGICAL TIMESCALE
def l_bio(a,b,c): #stellar lifetime is Gyr
    return 1.2*a**(8/5)*b**(-1/5)*c**(-4/5)
def f_bio(l,a,b,c):  
    return heaviside(l_bio(a,b,c)-l)

#PHOTOSYNTHESIS
def l_fizzle(a,b,c,w): #star too red: w is longest wavelength in nm
    return .21*(w/1100)**(-40/19)*a**(60/19)*b**(20/19)*c**(-10/19)
def l_fry(a,b,c,w): #star too blue: w is shortest wavelength in nm
    return 1.75*(w/400)**(-40/19)*a**(60/19)*b**(20/19)*c**(-10/19)
def f_photo(l,a,b,c): 
    return [1,(heaviside(l-l_fizzle(a,b,c,1100))*heaviside(l_fry(a,b,c,400)-l)),\
            (heaviside(l-l_fizzle(a,b,c,750))*heaviside(l_fry(a,b,c,600)-l))]

In [0]:
#FRACTION OF STARS WITH PLANETS

Z_inf=.011
def Z_min(l,a,b,c): #minimum metallicity for planet formation
    return 6.3*10**-4*(1.8*l)**(3/4)*a**-3*b**(-1/2)*c**(1/2)
def f_pstep(l,a,b,c): #fraction of stars with planets (step function)
    return heaviside(Z_inf-Z_min(l,a,b,c))
def f_sn(l,a,b,c): #fraction of galaxies that can retain sn ejecta
    return erfc(4.41*10**(-5)*a**2*b**(5/3))
def f_p(l,a,b,c): #fraction of stars with planets
    return [1,f_sn(l,a,b,c)*f_pstep(l,a,b,c)]

#HOT JUPITERS
def Z_maxpp(l,a,b,c): #maximum metallicity for no hot jupiters (planet-planet interactions)
    return .12/(1.8*l)*a**(13/6)*b**(3/2)
def f_hj2(l,a,b,c): #simplified fraction of earths without hot jupiters
    return [1,(1-(Z_inf/Z_maxpp(l,a,b,c))**2)*heaviside(Z_maxpp(l,a,b,c)-Z_inf)]

In [0]:
#NUMBER OF HABITABLE PLANETS

#ISOLATION MASS
qiso=1/3 #slope of planetesimal masses
p_sigma=0 # planet variance set by 0: entropy or 1: shot noise
p_ldep=1 # include stellar mass-planet mass correlation 0: no 1: yes
def r_iso(l,a,b,c): #ratio of isolation mass to terrrestrial mass
    return .1*(1.8*l)**2*a**-4*b**(-21/8)
def r_inn(l,a,b,c): #ratio of inner disk mass to isolation mass
    return 30*(1.8*l)**(-1/3)*a**(5/6)*b**(5/8)
def fiso(x): #fraction of terrestrial planets
    return min(1,(x/.3)**qiso)-min(1,(x/4)**qiso)
f_iso=np.vectorize(fiso)
def n_iso(x): # average number of planets
    return (qiso-1)/qiso*(x-x**(1-qiso))/(1-x**(1-qiso))

#GIANT IMPACT MASS
def r_olig(l,a,b,c): #ratio of oligarch mass to terrestrial mass
    return 2.64*(1.8*l)**(p_ldep*5/2)*a**(-9/2)*b**(-45/16) #accretion
#    return 2.64*(1.8*l)**(p_ldep*139/40)*a**(-39/4)*b**(-3)*c**(3/4) #irradiation
def f_olig(x,s): #number of terrestrial mass oligarchs
    return 1/2*(erf((np.log(4/x)+s**2/2)/np.sqrt(2)/s)\
                -erf((np.log(.3/x)+s**2/2)/np.sqrt(2)/s))
#    return np.exp(-np.pi/4*(.3/x)**2)-np.exp(-np.pi/4*(4/x)**2) #rayleigh dist
def sigma_m(l,a,b,c): #variance of m_olig
    return [1/np.sqrt(6),1/np.sqrt(r_inn(l,a,b,c))]
def n_olig(l,a,b,c): #average number of planets 
    return 3*(1.8*l)**(-5/6)*a**(4/3)*b**(13/16) #accretion
#    return 3*(1.8*l)**(-11/8)*a**(17/4)*b**(11/12)*c**(-5/12) #irradiation

def n_terr(l,a,b,c):
    return [1,n_olig(l,a,b,c)*f_olig(r_olig(l,a,b,c),sigma_m(l,a,b,c)[p_sigma]),n_iso(r_inn(l,a,b,c))*f_iso(r_iso(l,a,b,c))]

#TEMPERATE ZONE
def n_temp(l,a,b,c): #number of planets in temperate zone
    return .431*l**(-85/48)*a**(11/2)*b**(7/4)*c**(-5/8)
def temp_thresh(l,a,b,c): #temperate zone smaller than disk size
    return .01*(1.8*l)**(17/12)*a**(-5)*b**(-2)*c**(1/2)

In [0]:
#LIFE

n_hard=1 #number of hard steps
def r_time(l,a,b,c): #ratio of stellar to molecular timescale
    return l**(-5/2)*a**(4)*b**(-1/2)*c**-2
def r_area(a,b,c): #ratio of planet area to molecular area
    return a**(3/2)*b**(3/4)*c**-3 #L_mol~a_0
def S_tot(l,a,b,c): #entropy produced in stellar lifetime
    return (1.8*l)**(-119/40)*a**(17/2)*b**2*c**(-17/4)
def C_tot(l,a,b,c): #material limited biosphere
    return (1.8*l)**(-5/2)*a**(9/2)*b**(1/2)*c**(-3) 
def f_plates(a):
    return (heaviside(a_ptmax-a)*heaviside(a-a_ptmin))

#OXYGENATION TIME
r_o=.22 #Earth's ratio
def rip(x): #regulates the exp in rat_t
    return 500*np.tanh(x/500)
def rat_tup(l,a,b,c): #ratio of oxidation time to stellar lifetime
    return r_o*(1.8*l)**(81/40)*a**(-4)*b**(3/4)*c**(3/4)*\
    np.exp(rip(-18.85*(b**(-1/2)-1)+15.37*((1.8*l)**(-19/40)*a**(3/2)*b*c**(-1/4)-1)))
def rat_tdown(l,a,b,c): #ratio of timescales with drawdown
    return r_o*(1.8*l)**(5/2)*a**-3*b**(1/4)*np.exp(18.85*(1-b**(-1/2)))
def rat_t(l,a,b,c):
    return [0,rat_tdown(l,a,b,c),rat_tup(l,a,b,c),2/(1/(rat_tdown(l,a,b,c)+10**-6)+1/rat_tup(l,a,b,c))]

In [0]:
#DEATH

Myr=1
km=1
TW=1
def t_rec(l,a,b,c): #recovery time
#    return 10*a**(-2)*b**(-3/2)*Myr #mol
    return 10*(1.8*l)**(17/8)*a**(-15/2)*b**(-3)*c**(-1/4)*Myr #yr
def t_star(l,a,b,c): #stellar lifetime
    return 5500*(1.8*l)**(-5/2)*a**2*b**(-2)

#COMETS
def d_comet(l,a,b,c): #typical comet size
    return 1*(1.8*l)*a**(-25/9)*b**(-3/2)*c**(-5/6)*km
def d_sox(l,a,b,c): #extinction size (sulfates)
    return 1*(1.8*l)**(1/4)*a**(-3)*b**(-1)*c**(-1/2)*km
def d_dust(l,a,b,c): #extinction size (dust)
    return 1*(1.8*l)**(1/4)*a**(-5/3)*b**(-1)*c**(-1/2)/\
(1/2+1/2*erfc(.57*(.769+np.log(.01*a**(1/2)*b**(1/4)*c**(-1)))))**(1/3)*km
def d_both(l,a,b,c):
    return minn(d_sox(l,a,b,c),d_dust(l,a,b,c))
def G_comets(l,a,b,c): #comet extinction rate
    return (1.8*l)**(-16/5)*a**8*b**(-1/2)*(ramp(d_comet(l,a,b,c)/d_both(l,a,b,c)))**(1.5)/(3*Myr)/30

#GRBS
def vh(x): #volume function
    if x<=.075:
        return 40/3*x**3
    if .075<x<1:
        return x**2
    if 1<=x:
        return 1
vg=np.vectorize(vh)
def rat_grb(a,b,c): #ratio of grb death radius to galaxy radius 
    return 1/6*a**(-1)*b**(-3/2)*c**(-1/2)
def G_grb(l,a,b,c): #grb extinction rate
    return c*36*vg(rat_grb(a,b,c))/(90*Myr)

#GLACIATIONS
def Q_dirty(l,a,b,c): #dimensional analysis estimate of heat flux 
    return 47*a**(9/2)*b**(7/2)*c**(-1)*TW
def s_t(l,a,b,c): #dimensionless quantity in heat flux
    return .4242*a**(1/2)*b**(5/8)*c*(t_rec(l,a,b,c)/Myr)**(1/2)
def Q_form(l,a,b,c): #heat of formation
    return Q_dirty(l,a,b,c)*5.13/s_t(l,a,b,c)*math.e**(-s_t(l,a,b,c))
def f_rad(a): # dimensionless fraction in radioactive heat
    return math.e**(1+52.72*(1-(137/144/a)**(1/2))-math.e**(52.72*(1-(137/144/a)**(1/2))))
def Q_rad(l,a,b,c): #radioactive heat
    return 47*a**(3/2)*b**(3/4)*c**(-3)*f_rad(a)*TW
def Q_both(l,a,b,c):
    return (Q_form(l,a,b,c)+Q_rad(l,a,b,c))/2
def G_glac(l,a,b,c): #glaciation extinction rate
    return Q_both(l,a,b,c)/(47*TW)*a**(-7/2)*b**(-9/4)*c**3/(90*Myr)

#VOLCANOES
def G_vol(l,a,b,c): #volcano extinction rate
    return (Q_both(l,a,b,c)/(47*TW))**(1/2)*a**(-3/4)*b**(-3/8)*c**(3/2)/(90*Myr)

#TOTAL
def G_death(l,a,b,c): #total extinction rate
    return (p_comets*G_comets(l,a,b,c)+p_grb*G_grb(l,a,b,c)\
            +p_glac*G_glac(l,a,b,c)+p_vol*G_vol(l,a,b,c))/(p_comets+p_grb+p_glac+p_vol+10**-6)
def f_setback(t,G):
    return (1-t*G)*heaviside(1-t*G)
def f_IDH(t,G):
    return (t*G+10**-6)*(1-t*G)*heaviside(1-t*G)
def f_reset(t,ts,G):
    return 1-(1-np.exp(-10*t*G))**(ts/(10*t))
def f_int(l,a,b,c): #fraction of life that develops intelligence
    return [1,f_setback(t_rec(l,a,b,c),G_death(l,a,b,c)),\
            f_IDH(t_rec(l,a,b,c),G_death(l,a,b,c)),\
            f_reset(t_rec(l,a,b,c),t_star(l,a,b,c),G_death(l,a,b,c))]

In [0]:
#NUMBER OF OBSERVERS

def nobs(l,a,b,c,*P_list): #counts number of observers
    return pmeas(a,b,c)*Nstars(a,b,c)\
    *masch(l,a,b,c)**1\
    *f_photo(l,a,b,c)[p_photo]\
    *f_TL(l,a,b,c)**p_TL\
    *f_conv(l,a,b,c)**p_conv\
    *f_bio(l,a,b,c)**p_bio\
    *f_plates(a)**(p_plates)\
    *f_p(l,a,b,c)[1]\
    *f_hj2(l,a,b,c)[p_hj]\
    *n_terr(l,a,b,c)[p_terr]\
    *(n_temp(l,a,b,c)*heaviside(1-temp_thresh(l,a,b,c)))**p_temp\
    *r_time(l,a,b,c)**(n_hard*p_time)\
    *r_area(a,b,c)**(n_hard*p_area)\
    *S_tot(l,a,b,c)**(p_S)\
    *(1-rat_t(l,a,b,c)[p_O2])*heaviside(1-rat_t(l,a,b,c)[p_O2])\
    *f_int(l,a,b,c)[p_death]\
    
# SOBOL INTEGRATION
def init_sobol(): #initializes sobol integration parameters
    global Ns2,elp,at,ap,bt,ct,lt,ltp,lo,lc1
    Ns2=Ns
    elp=sobol_seq.i4_sobol_generate(4,Ns)
    at=elp[:,0]*(a_max-a_min)+a_min
    ap=elp[:,0]*(a_ptmax-a_ptmin)+a_ptmin
    bt=elp[:,1]*(b_max-b_min)+b_min
    ct=elp[:,2]*(c_max-c_min)+c_min
    lt=l_min(at,bt,ct)/elp[:,3]**(1/(beta_imf-1))
    ltp=l_min(ap,bt,ct)/elp[:,3]**(1/(beta_imf-1))
    lo=l_min(1,1,1)/elp[:,3]**(1/(beta_imf-1))

def probs(f): #calculates probability of our measurements
    def chance_a(l,a,b,c):
        return f(l,a,b,c)*heaviside(1-a)
    def chance_b(l,a,b,c):
        return f(l,a,b,c)*heaviside(1-b)
    def chance_c(l,a,b,c):
        return f(l,a,b,c)*heaviside(1-c)
    if 'Ns2' not in globals():
        init_sobol()
    elif Ns!=Ns2:
        init_sobol()
    if p_plates==1:
        au=ap
        lu=ltp
    else:
        au=at
        lu=lt
    num_a=np.mean(chance_a(lu,au,bt,ct))
    num_b=np.mean(chance_b(lu,au,bt,ct))
    num_c=np.mean(chance_c(lu,au,bt,ct))
    den=np.mean(f(lu,au,bt,ct))           
    pa,pb,pc=m1(num_a/den),m1(num_b/den),m1(num_c/den)
    Quo=[r3(pa),r3(pb),r3(pc)]
    if Q_l==1:
        Quo.append(probs_l(f))    
    if Q_avg==1:
        Quo.append(N_usvsavg(f))
    if Q_tO2==1:
        Quo.append(pO2u(f))
        Quo.append(pO2m(f))
    return Quo

def probs_l(f): #probability of being around a sunlike star within our universe
    num_l=np.mean(f(lo,1,1,1)*heaviside(1/1.8-lo))
    den_l=np.mean(f(lo,1,1,1))
    pl=1-num_l/den_l
    return r3(pl)

def N_usvsavg(f): #number of observers in our universe compared to average
    if p_plates==1:
        au=ap
        lu=ltp
    else:
        au=at
        lu=lt
    f_obs=np.mean(heaviside(f(lu,au,bt,ct)))
    return r3(np.mean(f(lo,1,1,1))/np.mean(f(lu,au,bt,ct))*f_obs)

def pO2u(f): #probability of our t_O2/t_star in universe
    num_g=np.mean(f(lo,1,1,1)*heaviside(r_o-rat_t(lo,1,1,1)[p_O2]))
    den_g=np.mean(f(lo,1,1,1)*heaviside(1-rat_t(lo,1,1,1)[p_O2]))
    return r3(1-num_g/den_g)

def pO2m(f): #probability of our t_O2/t_star in multiverse
    if p_plates==1:
        au=ap
        lu=ltp
    else:
        au=at
        lu=lt
    num_g=np.mean(f(lu,au,bt,ct)*heaviside(r_o-rat_t(lu,au,bt,ct)[p_O2]))
    den_g=np.mean(f(lu,au,bt,ct)*heaviside(1-rat_t(lu,au,bt,ct)[p_O2]))
    return r3(1-num_g/den_g)

def HH(*P_list):
   def nub(l,a,b,c):
       return nobs(l,a,b,c,*P_list)
   return probs(nub)

In [0]:
#TEST MULTIPLE HYPOTHESES ON RUNNING

Ns=10**5 #number of samples. 10**5 is decent, 10**6 is accurate, 10**7 professional

Q_l=1 #calculates p(M_sun) if =1
Q_avg=0 #caluclates N_obs/<N> if =1
Q_tO2=0 #calculates p(t_O2/t_star) if =1

print('[ p(alpha) , p(beta) , p(gamma) '\
        +', p(lambda) '*Q_l+', N_0/<N> '*Q_avg\
        +', p(tO2/tstar)_u , p(tO2/tstar)_m '*Q_tO2+']')

#TOGGLES
# unless otherwise specified, 0: off, 1: on
H_photo=[0,2] # 1: photo, 2: yellow
H_TL=[0,1]
H_conv=[0]
H_bio=[0]
H_plates=[0]
H_hj=[0]
H_terr=[0] # 1: giant impact 2: isolation
H_temp=[0]
H_time=[0]
H_area=[0]
H_S=[1]
H_O2=[0] # 1: drawdown, 2: drawup 3: both
H_death=[0] # 1: setback, 2: IDH, 3: reset
H_comets=[0]
H_grbs=[0]
H_glac=[0]
H_vol=[0]
H_guest=[0] # to quickly add another criterion. Toggles 0-1-2
# to add a new variable, need to change Hs, H_list, P_primes, and ps in P_list

H_list=[H_photo,H_TL,H_conv,H_bio,H_plates,H_hj,H_terr,H_temp,H_time,H_area,H_S,\
        H_O2,H_death,H_comets,H_grbs,H_glac,H_vol,H_guest]
P_primes=[['      ',' photo','yellow'],\
          ['   ',' TL'],\
          ['     ',' conv'],\
          ['    ',' bio'],\
          ['       ',' plates'],\
          ['   ',' HJ','hj-pd'],\
          ['    ','  GI','  iso'],\
          ['     ',' temp'],\
          ['     ',' time'],\
          ['     ',' area'],\
          ['  ',' S'],\
          ['       ',' O2down','   O2up',' O2both'],\
          ['        ',' setback','     IDH','   reset'],\
          ['       ',' comets',],\
          ['     ',' grbs'],\
          ['     ',' glac'],\
          ['    ',' vol'],\
          ['      ',' guest',' tseug']]

for P_list in itertools.product(*H_list):
    [p_photo,p_TL,p_conv,p_bio,p_plates,p_hj,p_terr,p_temp,\
     p_time,p_area,p_S,p_O2,p_death,p_comets,p_grb,p_glac,p_vol,p_guest]=P_list
    signs(*P_list)
    print(HH(*P_list))

[ p(alpha) , p(beta) , p(gamma) , p(lambda) ]
                                                S                                          
[0.0852, 0.0181, 0.0574, 0.000131]
       TL                                       S                                          
[0.41, 0.473, 0.318, 0.53]
yellow                                          S                                          
[0.184, 0.49, 0.411, 0.424]
yellow TL                                       S                                          
[0.28, 0.427, 0.465, 0.44]
